In [1]:
%load_ext autoreload
%autoreload 2

import logging
import requests
import json
import os
import time
import pandas as pd
import numpy as np

wd = os.path.abspath("__file__").replace("/__file__", "").replace("notebooks", "")
os.chdir(wd)

import requests
from datetime import datetime, timedelta, date
from logging.handlers import TimedRotatingFileHandler
# from binance.api import Binance
from src.utils import get_jinja_yaml_conf, create_db_engine, Postgres_connect, send_message_to_discord
from src.api import *
from src.processing import *

now = datetime.now()

In [2]:
conf = get_jinja_yaml_conf('./conf/webhook.yml', './conf/logging.yml', './conf/data.yml')
end_date = datetime.now().date() 

stream = logging.StreamHandler()
stream.setLevel(logging.DEBUG)
# logger 설정
logger = logging.getLogger('main')
logging.basicConfig(level=eval(conf['logging']['level']),
    format=conf['logging']['format'],
    handlers = [TimedRotatingFileHandler(filename =  conf['logging']['file_name'],
                                when=conf['logging']['when'],
                                interval=conf['logging']['interval'],
                                backupCount=conf['logging']['backupCount']), #logging.StreamHandler()
                                   stream]
                )


In [3]:
# Only for notebooks
import re

os.environ['_ts'] = datetime.astimezone(datetime.now()).strftime('%Y-%m-%d %H:%M:%S %z')

with open('./conf/credentials', "r") as file:
    # 각 라인 읽기
    for line in file:
        # 주석(#) 또는 빈 줄은 무시
        if line.strip() == '' or line.startswith('#'):
            continue

        # 각 라인을 '='를 기준으로 key와 value로 분리
        key, value = line.strip().split('=', 1)

        # $ENV 형식의 환경변수가 있을 경우 해당 값을 가져와서 설정
        env_var_pattern = re.compile(r'\$(\w+)')
        matches = env_var_pattern.findall(value)
        for match in matches:
            value = value.replace(f"${match}", os.environ.get(match, "")).replace('"', '')

        # 환경변수로 설정
        os.environ[key] = value

In [4]:
engine = create_db_engine(os.environ)
postgres_conn = Postgres_connect(engine)
api = Coingecko()

2024-04-09 16:41:57,279 (utils.py 75) INFO ::: Connect to 172.20.10.3. DB_NAME is coins
2024-04-09 16:41:57,280 (api.py 23) INFO ::: ### Coingecko main api is initialized! ###


In [16]:
# headers = {'x-cg-pro-api-key': os.environ['secret_key']}
total_page = 5

data = api.get_daily_coin_info(params = conf['detail']['markets']['params'], 
                               rename = conf['detail']['markets']['rename'], 
                               max_pages = conf['detail']['markets']['max_pages'])


2024-03-31 12:55:37,380 (4184423800.py 7) INFO ::: 1/5 starts!


NameError: name 'headers' is not defined

In [127]:
data['roi'] = data['roi'].map(lambda x: json.dumps(x) if type(x) is dict else x)
data['volume_rank'] = data['total_volume'].rank(ascending = False)

data['rank_prop'] = data['market_cap_rank'] / data['volume_rank'] 
data['vol_cap_ratio'] = data['total_volume'] / data['market_cap'] 

data.sort_values('vol_cap_ratio', ascending = False, inplace = True)
data_processed = data[[column for column in data.columns if column in conf['detail']['markets']['rename']]].reset_index(drop = True).round(2).reset_index()
data_processed['index'] += 1
data_processed.rename(columns = conf['detail']['markets']['rename'], inplace = True)

In [128]:
send_message_to_discord(os.environ['webhook_url'], f"{now.strftime('%Y년 %m월 %d일 %H시')} 코인 정보")
for i in range(conf['webhook_total_rows'] // conf['webhook_print_rows']):

    msg = [''.join(f"{column}{' '*(conf['webhook_space_len'] - len(str(column)*2))}" for column in data_processed.columns)]
    
    for n, vals in data_processed.iloc[i*conf['webhook_print_rows']:(i+1)*conf['webhook_print_rows']].iterrows():
        msg.append(''.join([f"{str(val)}{' '*(conf['webhook_space_len'] - len(str(val)))}" for val in vals]))
    msg = '\n'.join(msg)

    send_message_to_discord(os.environ['webhook_url'], f"```{msg}```")

2024-03-30 19:09:15,244 (utils.py 55) INFO ::: 메시지가 성공적으로 전송되었습니다.
2024-03-30 19:09:15,593 (utils.py 55) INFO ::: 메시지가 성공적으로 전송되었습니다.
2024-03-30 19:09:16,013 (utils.py 55) INFO ::: 메시지가 성공적으로 전송되었습니다.
2024-03-30 19:09:16,450 (utils.py 55) INFO ::: 메시지가 성공적으로 전송되었습니다.
2024-03-30 19:09:16,906 (utils.py 55) INFO ::: 메시지가 성공적으로 전송되었습니다.
2024-03-30 19:09:17,306 (utils.py 55) INFO ::: 메시지가 성공적으로 전송되었습니다.


In [125]:
datetime.now()

datetime.datetime(2024, 4, 1, 14, 48, 34, 789871)

In [59]:
conf_trending = conf['data']['trending']

data = json.loads(
    api.get_api_data(conf_trending['url']).text)

In [60]:

for key in conf_trending['data'].keys():
    keys = [key['item'] if 'item' in key else key for key in data[key]]

    for idx, _ in enumerate(keys):
        key_data = keys[idx]['data']
        keys[idx].update(key_data)
        del keys[idx]['data']

    keys_data = pd.DataFrame(keys)
    keys_data[conf_trending['data'][key]['json_cols']] = keys_data[conf_trending['data'][key]['json_cols']].map(lambda x: json.dumps(x) if type(x) is dict else x)
    keys_data['_ts'] = datetime.astimezone(datetime.now())
    postgres_conn.insert_df(keys_data, conf_trending['data'][key]['schema_name'], conf_trending['data'][key]['table_name'])

2024-04-02 14:39:02,217 (utils.py 145) INFO ::: Upload data successfully (rows: 15).
2024-04-02 14:39:02,221 (utils.py 145) INFO ::: Upload data successfully (rows: 7).
2024-04-02 14:39:02,225 (utils.py 145) INFO ::: Upload data successfully (rows: 6).


In [55]:
a[0]

,id,coin_id,name,symbol,market_cap_rank,thumb,small,large,slug,price_btc,score,price,price_change_percentage_24h,market_cap,market_cap_btc,total_volume,total_volume_btc,sparkline,content
0,catcoin-cash,25279,Catcoin,CAT,830,https://assets.coingecko.com/coins/images/2527...,https://assets.coingecko.com/coins/images/2527...,https://assets.coingecko.com/coins/images/2527...,catcoin,0.0000000000000123662896941389,0,8.611699e-10,"{""aed"": -6.221700192577507, ""ars"": -5.68315399...","$43,421,194",621.777183303026,"$5,113,701",73.4320855820086,https://www.coingecko.com/coins/25279/sparklin...,None
1,constellation-labs,4645,Constellation,DAG,362,https://assets.coingecko.com/coins/images/4645...,https://assets.coingecko.com/coins/images/4645...,https://assets.coingecko.com/coins/images/4645...,constellation,0.000000846897097632003,1,5.897414e-02,"{""aed"": -5.901839406782416, ""ars"": -5.36376894...","$176,631,892",2529.31045539928,"$1,233,075",17.7075584353917,https://www.coingecko.com/coins/4645/sparkline...,None
2,credefi,21396,Credefi,CREDI,984,https://assets.coingecko.com/coins/images/2139...,https://assets.coingecko.com/coins/images/2139...,https://assets.coingecko.com/coins/images/2139...,credefi,0.000000629884692238287,2,4.386236e-02,"{""aed"": 7.036927624659049, ""ars"": 7.6489843229...","$31,273,138",447.821018430575,"$4,177,124",59.9855180427537,https://www.coingecko.com/coins/21396/sparklin...,None
3,honeyland-honey,29256,Honeyland,HXD,893,https://assets.coingecko.com/coins/images/2925...,https://assets.coingecko.com/coins/images/2925...,https://assets.coingecko.com/coins/images/2925...,honeyland,0.00000229912322021562,3,1.603014e-01,"{""aed"": 8.678673814241087, ""ars"": 9.2751558882...","$37,321,014",534.654723787052,"$228,642",3.27929937479048,https://www.coingecko.com/coins/29256/sparklin...,None
4,gala,12493,GALA,GALA,56,https://assets.coingecko.com/coins/images/1249...,https://assets.coingecko.com/coins/images/1249...,https://assets.coingecko.com/coins/images/1249...,gala,0.000000912086039753763,4,6.351361e-02,"{""aed"": -7.757795585959874, ""ars"": -7.23033781...","$2,399,634,470",34361.9743653163,"$213,485,688",3065.75737856939,https://www.coingecko.com/coins/12493/sparklin...,"{""title"": ""What is GALA?"", ""description"": ""Gal..."
5,artrade,24894,Artrade,ATR,479,https://assets.coingecko.com/coins/images/2489...,https://assets.coingecko.com/coins/images/2489...,https://assets.coingecko.com/coins/images/2489...,artrade,0.0000010833850738861,5,7.544211e-02,"{""aed"": -7.7492838103411845, ""ars"": -7.2217773...","$120,275,058",1721.68805200718,"$5,553,999",79.7581058940175,https://www.coingecko.com/coins/24894/sparklin...,None
6,propbase,32932,Propbase,PROPS,681,https://assets.coingecko.com/coins/images/3293...,https://assets.coingecko.com/coins/images/3293...,https://assets.coingecko.com/coins/images/3293...,propbase,0.00000260737956911222,6,1.815663e-01,"{""aed"": -11.284118241602336, ""ars"": -10.776824...","$60,993,186",873.159725594297,"$1,594,644",22.899863197562,https://www.coingecko.com/coins/32932/sparklin...,None
7,mochi-thecatcoin,33083,Mochi,MOCHI,820,https://assets.coingecko.com/coins/images/3308...,https://assets.coingecko.com/coins/images/3308...,https://assets.coingecko.com/coins/images/3308...,mochi-thecatcoin,0.000000000633937341449245,7,4.414681e-05,"{""aed"": -13.582299947766543, ""ars"": -13.101566...","$44,098,045",631.741348112358,"$1,499,258",21.5289756763213,https://www.coingecko.com/coins/33083/sparklin...,None
8,tectum,29575,Tectum,TET,323,https://assets.coingecko.com/coins/images/2957...,https://assets.coingecko.com/coins/images/2957...,https://assets.coingecko.com/coins/images/2957...,tectum,0.000429711292481088,8,2.992318e+01,"{""aed"": -7.163907235639999, ""ars"": -6.63305350...","$220,747,499",3160.0101733433,"$2,023,824",29.063090547023,https://www.coingecko.com/coins/29575/sparklin...,None
9,dynex,27776,Dynex,DNX,654,https://assets.coingecko.com/coins/images/2777...,https://assets

In [82]:
data['categories']

[{'id': 227,
  'name': 'Dog-Themed Coins',
  'market_cap_1h_change': 0.8561431967520768,
  'slug': 'dog-themed-coins',
  'coins_count': 58,
  'data': {'market_cap': 57599844425.724525,
   'market_cap_btc': 827219.6423432477,
   'total_volume': 5347896856.757947,
   'total_volume_btc': 76803.77211505361,
   'market_cap_change_percentage_24h': {'aed': 0.19855735310348077,
    'ars': 0.3942295795028329,
    'aud': 0.21984566995657245,
    'bch': -10.622371389713903,
    'bdt': 0.288960034689263,
    'bhd': 0.2896986184493969,
    'bmd': 0.19842092849109266,
    'bnb': 2.9400156080143343,
    'brl': 0.1884314543994897,
    'btc': 0.22149517192529244,
    'cad': 0.4224108845989903,
    'chf': 0.18598091511791123,
    'clp': 0.2630705704226576,
    'cny': 0.33441114067263916,
    'czk': 0.5133176432993555,
    'dkk': 0.30526947539865895,
    'dot': 2.145405159964435,
    'eos': 2.025665389601573,
    'eth': -1.0216728730969133,
    'eur': 0.371240822955563,
    'gbp': 0.3125334914665203,
   

In [75]:
data['nfts']

[{'id': 'cryptocubes',
  'name': 'CryptoCubes',
  'symbol': 'CC',
  'thumb': 'https://assets.coingecko.com/nft_contracts/images/1108/standard/cryptocubes.png?1707287648',
  'nft_contract_id': 1108,
  'native_currency_symbol': 'eth',
  'floor_price_in_native_currency': 8.0,
  'floor_price_24h_percentage_change': 161.738321799007,
  'data': {'floor_price': '8.00 ETH',
   'floor_price_in_usd_24h_percentage_change': '161.738321799007',
   'h24_volume': '3.00 ETH',
   'h24_average_sale_price': '3.00 ETH',
   'sparkline': 'https://www.coingecko.com/nft/1108/sparkline.svg',
   'content': None}},
 {'id': 'lasercat-nft',
  'name': 'LaserCat NFT',
  'symbol': 'Cat',
  'thumb': 'https://assets.coingecko.com/nft_contracts/images/1142/standard/lasercat-nft.png?1707287668',
  'nft_contract_id': 1142,
  'native_currency_symbol': 'eth',
  'floor_price_in_native_currency': 12.6,
  'floor_price_24h_percentage_change': 58.8732882799631,
  'data': {'floor_price': '12.60 ETH',
   'floor_price_in_usd_24h_pe

In [73]:
pd.DataFrame(data['categories'])

,id,name,market_cap_1h_change,slug,coins_count,data
0,227,Dog-Themed Coins,0.856143,dog-themed-coins,58,"{'market_cap': 57599844425.724525, 'market_cap..."
1,274,Layer 3 (L3),0.814780,layer-3-l3,4,"{'market_cap': 1186718717.0325294, 'market_cap..."
2,113,Meme,0.591705,meme-token,567,"{'market_cap': 68187540950.435646, 'market_cap..."
3,88,Metaverse,0.428118,metaverse,160,"{'market_cap': 17008626681.073338, 'market_cap..."
4,231,DWF Labs Portfolio,0.420717,dwf-labs-portfolio,157,"{'market_cap': 68465703811.6196, 'market_cap_b..."
5,170,Kommunitas Launchpad,0.337853,kommunitas,28,"{'market_cap': 53784995.85207335, 'market_cap_..."


In [137]:
p['coin_id'] = 'bitcoin'
p = pd.concat([p,
pd.DataFrame(p['companies'].tolist()).rename(conf['data']['companies_treasury']['company_rename'], axis = 1)], axis = 1)
p.drop('companies', axis = 1, inplace = True)
db_conn.insert_df(p, schema_name = conf['data']['companies_treasury']['schema_name'], table_name = conf['data']['companies_treasury']['table_name'])

NameError: name 'db_conn' is not defined

In [138]:
p

,total_holdings,total_value_usd,market_cap_dominance,_ts,coin_id,name,symbol,country,company_holdings,company_entry_value_usd,company_current_value_usd,percentage_of_company_supply
0,264136.0,1.836817e+10,1.35,2024-04-01 13:55:27.318413+09:00,bitcoin,MicroStrategy Inc.,NASDAQ:MSTR,US,174530,4680000000,12136916803,0.831
1,264136.0,1.836817e+10,1.35,2024-04-01 13:55:27.318413+09:00,bitcoin,Galaxy Digital Holdings,TSE: GLXY,US,17518,0,1218211817,0.083
2,264136.0,1.836817e+10,1.35,2024-04-01 13:55:27.318413+09:00,bitcoin,Marathon Digital Holdings,NASDAQ:MARA,US,13716,189087000,953818546,0.065
3,264136.0,1.836817e+10,1.35,2024-04-01 13:55:27.318413+09:00,bitcoin,"Tesla, Inc.",NASDAQ:TSLA,US,10500,336000000,730176052,0.050
4,264136.0,1.836817e+10,1.35,2024-04-01 13:55:27.318413+09:00,bitcoin,Hut 8 Mining Corp,NASDAQ:HUT,CA,9366,0,651317038,0.045
5,264136.0,1.836817e+10,1.35,2024-04-01 13:55:27.318413+09:00,bitcoin,"Coinbase Global, Inc",NASDAQ:COIN,US,9181,207783800,638452032,0.044
6,264136.0,1.836817e+10,1.35,2024-04-01 13:55:27.318413+09:00,bitcoin,Block Inc.,NYSE:SQ,US,8027,220000000,558202206,0.038
7,264136.0,1.836817e+10,1.35,2024-04-01 13:55:27.318413+09:00,bitcoin,"Riot Platforms, Inc",NASDAQ:RIOT,US,7327,0,509523803,0.035
8,264136.0,1.836817e+10,1.35,2024-04-01 13:55:27.318413+09:00,bitcoin,Hive Blockchain,NASDAQ:HIVE,CA,2596,0,180527336,0.012
9,264136.0,1.836817e+10,1.35,2024-04-01 13:55:27.318413+09:00,bitcoin,CleanSpark Inc.,NASDAQ:CLSK,US,2575,0,179066984,0.012


In [130]:
data = p

data = pd.DataFrame([data['data']])
data['_ts'] = datetime.now()
data['market_cap_percentage'] = data['market_cap_percentage'].map(lambda x: json.dumps(x) if type(x) is dict else x)
data['total_market_cap'] = data['total_market_cap'].map(lambda x: json.dumps(x) if type(x) is dict else x)
data['total_volume'] = data['total_volume'].map(lambda x: json.dumps(x) if type(x) is dict else x)
postgres_conn.insert_df(data, schema_name = 'coingecko', table_name = 'global')

2024-04-01 15:14:54,509 (utils.py 145) INFO ::: Upload data successfully (rows: 1).


In [31]:
data2[data2['id'] == 'aave-tokens']

,id,name,market_cap,market_cap_change_24h,content,top_3_coins,volume_24h,updated_at,_ts
169,aave-tokens,Aave Tokens,0.0,0.0,None,[https://assets.coingecko.com/coins/images/116...,2.710641e+06,2024-04-01T03:40:32.718Z,2024-04-01 12:37:20.255858+09:00


In [18]:
data[data['name'].duplicated(keep = False)]

,id,chain_identifier,name,shortname,native_coin_id,_ts


In [83]:
data = api.get_daily_coin_info(start_pages = 18, max_pages = 20, params = conf['data']['daily_coin_info']['params'])

2024-03-31 21:54:56,796 (api.py 79) INFO ::: Get daily coin info (18/20)
2024-03-31 21:54:57,601 (api.py 79) INFO ::: Get daily coin info (19/20)
2024-03-31 21:54:57,883 (api.py 79) INFO ::: Get daily coin info (20/20)


In [92]:
preprocessing_daily_coin_info(data)

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,_ts,volume_rank,rank_prop,vol_cap_ratio
224,aave-aave,aaave,Aave AAVE,https://assets.coingecko.com/coins/images/1724...,1.261300e+02,0.0,None,16434836.0,3.145310e+03,1.291300e+02,...,2021-08-17T12:17:43.687Z,4.621000e+01,1.732990e+02,2022-06-19T06:02:50.088Z,None,2024-03-31T12:46:57.535Z,2024-03-31 21:05:48.652713+09:00,338.0,NaN,inf
0,madlad,mad,MADLAD,https://assets.coingecko.com/coins/images/3614...,6.771000e-04,0.0,None,656923.0,5.338700e+04,7.895900e-04,...,2024-03-18T05:02:53.525Z,5.959400e-04,1.361952e+01,2024-03-31T09:14:27.623Z,None,2024-03-31T12:47:56.132Z,2024-03-31 21:05:48.652713+09:00,167.0,NaN,inf
1,shield-network,shieldnet,Shield Network,https://assets.coingecko.com/coins/images/1490...,2.965600e-08,0.0,None,29125.0,4.158410e-03,2.997400e-08,...,2021-05-03T18:47:34.477Z,1.200000e-16,2.464378e+10,2021-06-08T12:28:46.507Z,None,2024-03-30T21:37:04.680Z,2024-03-31 21:05:48.652713+09:00,693.0,NaN,inf
2,catwifbag,bag,catwifbag,https://assets.coingecko.com/coins/images/3612...,5.597300e-02,0.0,None,55973246.0,7.699572e+06,8.699500e-02,...,2024-03-25T16:39:14.310Z,5.089460e-03,1.003921e+03,2024-03-19T01:35:10.808Z,None,2024-03-31T12:50:35.018Z,2024-03-31 21:05:48.652713+09:00,15.0,NaN,inf
3,tradix,tx,Tradix,https://assets.coingecko.com/coins/images/2899...,3.050190e-03,0.0,None,305223.0,1.969710e+03,3.085540e-03,...,2024-03-24T15:51:49.033Z,1.299480e-03,1.343890e+02,2024-03-25T18:35:35.334Z,None,2024-03-31T12:50:30.846Z,2024-03-31 21:05:48.652713+09:00,363.0,NaN,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,aave-amm-weth,aammweth,Aave AMM WETH,https://assets.coingecko.com/coins/images/1725...,3.616800e+03,0.0,None,934328.0,0.000000e+00,3.630340e+03,...,2021-11-10T15:08:43.779Z,8.967600e+02,3.032470e+02,2022-06-18T21:00:56.854Z,None,2024-03-31T12:51:16.151Z,2024-03-31 21:05:48.652713+09:00,724.5,NaN,NaN
246,aave-bal,abal,Aave BAL,https://assets.coingecko.com/coins/images/1724...,5.170000e+00,0.0,None,342946.0,0.000000e+00,5.230000e+00,...,2021-09-04T13:07:07.586Z,2.950000e+00,7.541366e+01,2023-10-19T03:00:05.284Z,None,2024-03-31T12:51:18.823Z,2024-03-31 21:05:48.652713+09:00,724.5,NaN,NaN
247,aave-balancer-pool-token,abpt,Aave Balancer Pool Token,https://assets.coingecko.com/coins/images/1678...,1.875670e-01,0.0,None,NaN,0.000000e+00,1.910370e-01,...,2021-09-05T21:07:18.735Z,6.261400e-02,1.998643e+02,2022-06-18T21:00:55.668Z,None,2024-03-31T12:46:01.758Z,2024-03-31 21:05:48.652713+09:00,724.5,NaN,NaN
248,aave-bat,abat,Aave BAT,https://assets.coingecko.com/coins/images/1424...,3.310160e-01,0.0,None,223669.0,0.000000e+00,3.370520e-01,...,2021-11-28T01:02:09.902Z,1.594330e-01,1.075842e+02,2023-09-11T21:00:06.635Z,None,2024-03-31T12:51:21.330Z,2024-03-31 21:05:48.652713+09:00,724.5,NaN,NaN


In [68]:
data

""


In [110]:
postgres_conn.insert_df(preprocessing_daily_coin_info(data), schema_name = 'coingecko', table_name = 'daily_coin_info')

2024-03-31 22:05:36,012 (utils.py 145) INFO ::: Upload data successfully (rows: 704).


In [106]:
data.drop_duplicates('id', keep = 'first', inplace = True)

In [103]:
data[data['id'] == 'mobist']

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,_ts
230,mobist,mitx,Mobist,https://assets.coingecko.com/coins/images/2387...,0.000374,0.0,None,3739669.0,45.63,0.000404,...,NaN,0.032227,-98.83998,2022-04-22T12:04:47.707Z,0.00021,77.84805,2022-07-07T15:53:49.298Z,None,2024-03-31T12:43:06.317Z,2024-03-31 21:05:48.652713+09:00
10,mobist,mitx,Mobist,https://assets.coingecko.com/coins/images/2387...,0.000374,0.0,None,3739669.0,45.63,0.000404,...,NaN,0.032227,-98.83998,2022-04-22T12:04:47.707Z,0.00021,77.84805,2022-07-07T15:53:49.298Z,None,2024-03-31T12:43:06.317Z,2024-03-31 21:05:48.652713+09:00
